# NRCS Version of Jupyter SMV widget

In [2]:
import os
import json
import warnings
from math import ceil
from io import StringIO
from datetime import datetime

import requests
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import shape

import folium
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from IPython.display import display
from ipywidgets import Layout, Button, IntProgress, Output, HBox, VBox, HTML, interactive, SelectionRangeSlider

disabled_sources =  ["FLUXNET", "MODIS", "GRACE", "PBOH2O"]
ignore_variables = ["sample","time","stat","lat","lon"]

# ----------------------------------------------------------------------------
# app settings

#font = {'family': 'normal', 'weight': 'normal', 'size': 8}
#plt.rc('font', **font)

smvdownload = "https://airmoss.ornl.gov/cgi-bin/viz/api/download.pl?"
smvdatasets = pd.read_csv(
    "docs/smvdatasets.csv", 
    index_col="dataset", 
    header=0)

numvalid = lambda v: np.count_nonzero(~np.isnan(v.data))
allnan = lambda v: numvalid(v)==0

latatts = dict(
    standard_name="latitude",
    long_name="sample latitude",
    units="degrees_north")

lonatts = dict(
    standard_name="latitude",
    long_name="sample latitude",
    units="degrees_north")

# ease data ------------------------------------------------------------------
latf = "docs/EASE2_M09km.lats.3856x1624x1.double"
lonf = "docs/EASE2_M09km.lons.3856x1624x1.double"

lats = np.fromfile(latf, dtype=np.float64).flatten() 
lons = np.fromfile(lonf, dtype=np.float64).flatten()
crds = np.dstack((lats,lons))[0]

sample_header = ["id", "lat", "lon", "samp"]
layer_header = ["id", "lat", "lon", "layer", "samples", "points", "xr"]

download_msg = """<p style="text-align:center;">Click <b>Submit</b> to downloa
d Soil Moisture Visualizer data for this site.<br></p>"""

# ----------------------------------------------------------------------------

def get_colors(n, cmap=cm.Set2):
    """ 
    Takes integer count of colors to map and optional kwarg: 
      'cmap=matplotlib.cm.<cmap>'.
    """

    cspace = np.linspace(0.0, 1.0, n)           # *1
    rgb = cmap(cspace)                          # *2
    cols = [colors.to_hex(c[0:3]) for c in rgb] # *3

    return(cols)


# ----------------------------------------------------------------------------
# SMV sample dataset formatters


def txt_to_pd(response_text):
    """Parses response.text to data frame with date index."""
    
    f = StringIO(response_text)                      # get file from string

    df = pd.read_csv(
        f, 
        header=4, 
        index_col="time",
        error_bad_lines=False, 
        warn_bad_lines=False)                        # read to df
    df.index = pd.to_datetime(df.index)              # convert index to dates
    
    return(df)


def split_pd(col):
    """Splits pd column by ; and set all values to float, nan."""
    
    df = col.str.split(";",n=2,expand=True)           # split col by ;
    df = df.replace('', np.nan)                       # set '' to nan
    df = df.astype(float)                             # set all to float
    df.columns = ["Min","Mean","Max"]                 # add column names
    
    return(df)


def pd_to_xr(dataset, df):
    """Makes an xr.Dataset from a pandas column (series) and coords."""

    a = smvdatasets.loc[dataset].to_dict()
    x = xr.DataArray(df, name=dataset, attrs=a)
    x = x.rename(dict(dim_1="stat"))
    
    return(x)


def get_sample_xr(samp):
    """ """

    d = ["sample"]                          
    s = xr.DataArray(data=[samp.id], dims=d) # get sample, lat, lon xr arrays
    y = xr.DataArray(data=[samp.lat], coords=[s], dims=d, attrs=latatts)
    x = xr.DataArray(data=[samp.lon], coords=[s], dims=d, attrs=lonatts)

    df = samp.df                                         # get the sample df
    ds = {}
    for dataset in df.columns:
        a = smvdatasets.loc[dataset].to_dict()
        if a["source"] not in disabled_sources:
            split_column = split_pd(df[dataset])
            ds[dataset] = pd_to_xr(dataset, split_column)

    xds = xr.merge(ds.values())                          # merge to one xr
    xds = xds.assign_coords(lat=y, lon=x)                # add coord arrays
    
    return(xds)


def from_geojson(input_geojson):
    """ """
    with open(input_geojson, "r") as f:
        shapes = json.load(f)
    features = shapes["features"]
    cols = get_colors(len(features))
    return((features, cols))

In [14]:
def get_ease(shapely_geom):
    """ """

    bnds = shapely_geom.bounds 
    ease = crds[
        (bnds[1]<lats) & (lats<bnds[3]) &     # ybnds < lat < ybnds
        (bnds[0]<lons) & (lons<bnds[2])]      # xbnds < lon < xbnds

    ease_table = {"id":[], "lat":[], "lon":[], "in":[]}
    for i,p in enumerate(ease):
        
        ease_table["id"].append(i)
        ease_table["lat"].append(p[0])
        ease_table["lon"].append(p[1])
        
        shapely_pt = shape({                  # input to shapely.shape is a
            "type": "Point",                  # python dict equivalent of
            "coordinates": (p[1], p[0])})     # geojson point geometry
        
        if shapely_geom.contains(shapely_pt): # if point inside poly
            ease_table["in"].append(True)
        else:
            ease_table["in"].append(False)
        
    return(pd.DataFrame(ease_table))


def get_layer_data(i, feat, col="#FFFFFF", opac=0.4, samp=True):
    """ """

    shapely_geom = shape(feat["geometry"])              # shapely geom
    ease = get_ease(shapely_geom) if samp else None     # ease grid points
    cent = shapely_geom.centroid                        # centroid
    lat, lon = cent.y, cent.x                           # lat, lon
    feat["properties"].update({
        "id": i, 
        "style": {
            "weight": 0.75,
            "color": "aliceblue",
            "fillColor": col,
            "fillOpacity": opac}})

    return((feat, ease, lat, lon))

In [19]:
class Layer(object):


    def __init__(self, i, feat, col=None):
        """"""
        
        layer_data = get_layer_data(i, feat, col)
        
        self.id = i
        self.feat = layer_data[0]
        self.ease = layer_data[1]
        self.lat = layer_data[2]
        self.lon = layer_data[3]
        
        #self.layer = GeoJSON(
        #    data=self.feat,
        #    hover_style={
        #        "color": "white", 
        #        "fillOpacity": 0.8})
        #self.layer.on_click(self.toggle)

        self.dl = False    # downloaded or nah?
        self.on = False    # toggle on or nah?

    #def toggle(self, **kwargs):
    #    """Routine for when a new USFS polygon is selected."""
    #    if list(kwargs.keys()) != ['event', 'properties']: # check event
    #        return(None)                                   # skip basemap
    #    self.on = False if self.on else True               # update status
        
    def update(self, **kwargs):
        for arg, val in kwargs.items():
            setattr(self.layer, arg, val)

In [20]:
shapes = "docs/usfs_sites/Sites_lf_geo.json"

features, cols = from_geojson(shapes)        # get features and cols
poly = Layer(0, features[0])            # get Layer class

poly

In [21]:
poly.ease.head(5)

,id,lat,lon,in
0,0,32.080291,-110.025934,False
1,1,32.080291,-109.932573,False
2,2,32.080291,-109.839212,False
3,3,32.080291,-109.745851,False
4,4,32.080291,-109.652490,False


In [ ]:
features, cols = from_geojson(infeats)        # get features and cols

layers = []                                   # a temporary list 
for i, feat in enumerate(features):           # loop over features

    poly = Layer(i, feat, cols[i])            # get Layer class

    pts, samps = LayerGroup(), []             # points group; Samples
    for j, p in enumerate(poly.ease):         # loop EASE grid pts
        s = Sample(j, p[0], p[1])             # make Sample instance
        pts.add_layer(s.pt)                   # add to points group
        samps.append((j, p[0], p[1], s))      # append tuple to list  

    samps = pd.DataFrame(samps, columns=sample_header)       # samples
    layers.append((i,poly.lat,poly.lon,poly,samps,pts,None)) # append

self.layers = pd.DataFrame(layers, columns=layer_header)     # layers
self.selected = None


In [ ]:

bmap = mwg.basemap_to_tiles(mwg.basemaps.Esri.WorldImagery)    # map widget 

map_args = dict(
    center=(32.75, -109), 
    zoom=7, 
    scroll_wheel_zoom=True)

submit_args = dict(                                            # submit button
    description="Submit", 
    disabled=True, 
    button_style="success")

progress_args = dict(                                          # progress bar
    description="Progress: ", 
    layout=Layout(width="95%"))

# Original NASA-USFS Workshop widget

In [1]:
%matplotlib widget
from ursjupyter import *
from smvjupyter import *
warnings.filterwarnings('ignore')

usfs_sites = "docs/usfs_sites/Sites_lf_geo.json"                      # USFS sites
usfs_regions = "docs/usfs_admin/USFS_admin_boundaries.json"           # USFS administrative regions

'Login successful. Download with: session.get(url)'

In [2]:
app = JupyterSMV(usfs_sites, anc=usfs_regions, session=session)